In [ ]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import Sequential
from tensorflow import keras

In [ ]:
data = keras.datasets.fashion_mnist

In [ ]:
(train_img, train_label),(test_img, test_label) = data.load_data()

In [ ]:
# scaling the images
train_img = train_img/255.0
test_img = test_img/255.0

In [ ]:
train_img[0].shape

(28, 28)

In [ ]:
# in CNN, we have to reshape our images and should make 4dimensions
train_img = train_img.reshape(len(train_img), 28,28,1)
test_img = test_img.reshape(len(test_img), 28,28,1)

In [ ]:
train_img.shape

(60000, 28, 28, 1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
        keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
        keras.layers.Flatten(),
        keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
        keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


tuner_search.search(train_img,train_label,epochs=3,validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 128)               4718720   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4,757,578
Trainable params: 4,757,578
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_img, train_label, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1593 - accuracy: 0.9413 - val_loss: 0.2485 - val_accuracy: 0.9112
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1071 - accuracy: 0.9594 - val_loss: 0.2838 - val_accuracy: 0.9133
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0693 - accuracy: 0.9746 - val_loss: 0.3042 - val_accuracy: 0.9167
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0443 - accuracy: 0.9834 - val_loss: 0.3911 - val_accuracy: 0.9198
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0342 - accuracy: 0.9879 - val_loss: 0.4183 - val_accuracy: 0.9162
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0246 - accuracy: 0.9913 - val_loss: 0.4910 - val_accuracy: 0.9147
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0211 - accuracy: 0.9927 - val_loss: 0.4884 - val_accuracy